In [1]:
import griddata as gd

net, const_load_household, const_load_heatpump, time_steps, df_household_prognosis, df_season_heatpump_prognosis, heatpump_scaling_factors_df, T_amb = gd.setup_grid_powertech25(season='winter')


In [ ]:
print("Household ConstControl indices:", const_load_household.element_index)
print("Heat Pump ConstControl indices:", const_load_heatpump.element_index)

In [2]:
print(time_steps)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            158, 159, 160, 161, 162, 163, 164, 165, 166, 167],
           dtype='int64', length=168)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a directed graph
G = nx.DiGraph()

# Add all buses as nodes
for bus in net.bus.index:
    G.add_node(bus)

# Add edges based on line connections with edge index as an attribute
line_edges = {}
for idx, line in enumerate(net.line.itertuples()):
    from_bus = line.from_bus
    to_bus = line.to_bus
    edge = (from_bus, to_bus)
    G.add_edge(from_bus, to_bus)
    line_edges[edge] = f"Line {idx+1}"

# Add edges for transformer connections with edge index as an attribute
trafo_edges = {}
for idx, trafo in enumerate(net.trafo.itertuples()):
    hv_bus = trafo.hv_bus
    lv_bus = trafo.lv_bus
    edge = (hv_bus, lv_bus)
    G.add_edge(hv_bus, lv_bus)
    trafo_edges[edge] = f"Trafo {idx}"

# Combine all edge indices into one dictionary
edge_labels = {**line_edges, **trafo_edges}

# Find the root bus (e.g., the slack bus or transformer bus)
root_bus = net.ext_grid.bus.iloc[0]

# Debugging: Ensure the root bus exists in the graph
if root_bus not in G.nodes:
    print(f"Warning: Root bus {root_bus} is not in the graph.")
    print(f"Available nodes: {list(G.nodes)}")
else:
    print(f"Root bus {root_bus} found.")

# Generate a topological sort to ensure hierarchy
levels = {node: 0 for node in G.nodes}
visited = set()

def assign_levels(node, level):
    if node in visited:
        return
    visited.add(node)
    levels[node] = level
    for neighbor in G.neighbors(node):
        assign_levels(neighbor, level + 1)

# Assign levels starting from the root bus
assign_levels(root_bus, 0)

# Define a position based on levels with increased vertical spacing
pos = {}
nodes_by_level = {}
vertical_spacing = 15  # Increase vertical spacing to 15 units

for node, level in levels.items():
    if level not in nodes_by_level:
        nodes_by_level[level] = []
    nodes_by_level[level].append(node)

for level, nodes in nodes_by_level.items():
    y = -level * vertical_spacing  # Adjusted vertical spacing
    x_positions = range(len(nodes))
    for x, node in zip(x_positions, nodes):
        pos[node] = (x, y)

# Plot the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=500,
    node_color="lightblue",
    font_size=10,
    arrowsize=10,
)

# Highlight the root bus
nx.draw_networkx_nodes(G, pos, nodelist=[root_bus], node_color="red", node_size=700)

# Add edge labels
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)

plt.title("Hierarchical Representation of the Grid with Transformer Connections and Edge Indices")
plt.show()


In [ ]:
import pandas as pd

# Load the CSV file (replace 'your_file.csv' with the actual file path)
file_path = 'householdPrognosis.csv'
df = pd.read_csv(file_path, sep=';')

# Display the first few rows of the dataframe
print("First few rows of the data:")
print(df.head())

# Display the header (column names) of the dataframe
print("\nColumn names (Header):")
print(df.columns)

# Print the length of the 'dayOfWeek' column
print("\nLength of the 'dayOfWeek' column:")
print(df['meanP'].count())  # .count() counts non-null entries in the column

# Print the last two values in the 'dayOfWeek' column
print("\nLast two values in the 'dayOfWeek' column:")
print(df['meanP'].tail(2))

In [ ]:
import matplotlib.pyplot as plt
# Filter the data for rows where season is 'winter'
winter_data = df[df['season'] == 'winter']

# Convert the 'meanP' column to numeric, replacing commas with dots
winter_data['meanP'] = winter_data['meanP'].str.replace(',', '.').astype(float)

# Plot the 'meanP' column
plt.figure(figsize=(10, 6))
plt.plot(winter_data['meanP'], marker='o', linestyle='-', color='b')
plt.title('MeanP during Winter Season')
plt.xlabel('Index')
plt.ylabel('MeanP')
plt.grid(True)
plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a directed graph
G = nx.DiGraph()

# Add all buses as nodes
for bus in net.bus.index:
    G.add_node(bus)

# Add edges based on line connections with edge index as an attribute
line_edges = {}
for idx, line in enumerate(net.line.itertuples()):
    from_bus = line.from_bus
    to_bus = line.to_bus
    edge = (from_bus, to_bus)
    G.add_edge(from_bus, to_bus)
    line_edges[edge] = f"Line {idx}"

# Add edges for transformer connections with edge index as an attribute
trafo_edges = {}
for idx, trafo in enumerate(net.trafo.itertuples()):
    hv_bus = trafo.hv_bus
    lv_bus = trafo.lv_bus
    edge = (hv_bus, lv_bus)
    G.add_edge(hv_bus, lv_bus)
    trafo_edges[edge] = f"Trafo {idx}"

# Combine all edge indices into one dictionary
edge_labels = {**line_edges, **trafo_edges}

# Find the root bus (e.g., the slack bus or transformer bus)
root_bus = net.ext_grid.bus.iloc[0]

# Debugging: Ensure the root bus exists in the graph
if root_bus not in G.nodes:
    print(f"Warning: Root bus {root_bus} is not in the graph.")
    print(f"Available nodes: {list(G.nodes)}")
else:
    print(f"Root bus {root_bus} found.")

# Generate a topological sort to ensure hierarchy
levels = {node: 0 for node in G.nodes}
visited = set()

def assign_levels(node, level):
    if node in visited:
        return
    visited.add(node)
    levels[node] = level
    for neighbor in G.neighbors(node):
        assign_levels(neighbor, level + 1)

# Assign levels starting from the root bus
assign_levels(root_bus, 0)

# Define a position based on levels
pos = {}
nodes_by_level = {}
for node, level in levels.items():
    if level not in nodes_by_level:
        nodes_by_level[level] = []
    nodes_by_level[level].append(node)

for level, nodes in nodes_by_level.items():
    y = -level  # Downward hierarchy
    x_positions = range(len(nodes))
    for x, node in zip(x_positions, nodes):
        pos[node] = (x, y)

# Plot the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=500,
    node_color="lightblue",
    font_size=10,
    arrowsize=10,
)

# Highlight the root bus
nx.draw_networkx_nodes(G, pos, nodelist=[root_bus], node_color="red", node_size=700)

# Add edge labels
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)

plt.title("Hierarchical Representation of the Grid with Transformer Connections and Edge Indices")
plt.show()


In [5]:
import pandapower as pp
# Buses and lines to remove
buses_to_drop = [14, 34, 9, 19]

# Remove lines associated with the buses to drop
lines_to_drop = net.line[
    (net.line.from_bus.isin(buses_to_drop)) | (net.line.to_bus.isin(buses_to_drop))
].index

# Drop the identified lines
pp.drop_lines(net, lines_to_drop)

# Drop the identified buses
pp.drop_buses(net, buses_to_drop)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create a directed graph
G = nx.DiGraph()

# Add all buses as nodes
for bus in net.bus.index:
    G.add_node(bus)

# Add edges based on line connections
for line in net.line.itertuples():
    from_bus = line.from_bus
    to_bus = line.to_bus
    G.add_edge(from_bus, to_bus)

# Add edges for transformer connections
for trafo in net.trafo.itertuples():
    hv_bus = trafo.hv_bus
    lv_bus = trafo.lv_bus
    G.add_edge(hv_bus, lv_bus)

# Find the root bus (e.g., the slack bus or transformer bus)
root_bus = net.ext_grid.bus.iloc[0]

# Debugging: Ensure the root bus exists in the graph
if root_bus not in G.nodes:
    print(f"Warning: Root bus {root_bus} is not in the graph.")
    print(f"Available nodes: {list(G.nodes)}")
else:
    print(f"Root bus {root_bus} found.")

# Generate a topological sort to ensure hierarchy
levels = {node: 0 for node in G.nodes}
visited = set()

def assign_levels(node, level):
    if node in visited:
        return
    visited.add(node)
    levels[node] = level
    for neighbor in G.neighbors(node):
        assign_levels(neighbor, level + 1)

# Assign levels starting from the root bus
assign_levels(root_bus, 0)

# Define a position based on levels
pos = {}
nodes_by_level = {}
for node, level in levels.items():
    if level not in nodes_by_level:
        nodes_by_level[level] = []
    nodes_by_level[level].append(node)

for level, nodes in nodes_by_level.items():
    y = -level  # Downward hierarchy
    x_positions = range(len(nodes))
    for x, node in zip(x_positions, nodes):
        pos[node] = (x, y)

# Plot the graph
plt.figure(figsize=(12, 8))
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=500,
    node_color="lightblue",
    font_size=10,
    arrowsize=10,
)

# Highlight the root bus
nx.draw_networkx_nodes(G, pos, nodelist=[root_bus], node_color="red", node_size=700)
plt.title("Hierarchical Representation of the Grid with Transformer Connections")
plt.show()


In [94]:
# Set constant loads to 3 kW for all loads except R11 to R18
constant_loads = net.load.index.difference([1, 2, 3, 4, 5])  # Indices of loads R1, I2, C1, C12, C13, C14, C17, C18, C19, C20
net.load.loc[constant_loads, 'p_mw'] = 15/1000

In [95]:
# Initialize ConstControl controller to update values of the loads R11 to R18
profile_loads = net.load.index.intersection([0, 1, 2, 3, 4, 5])  # Indices of loads R11 to R18
const_load = control.ConstControl(net, element='load', element_index=profile_loads,
                                  variable='p_mw', data_source=ds, profile_name=["mult"]*len(profile_loads))


In [96]:
# Run a power flow for the initial condition to check if the network is properly configured
try:
    pp.runpp(net)
except pp.optimal_powerflow.OPFNotConverged:
    print("Initial power flow did not converge. Please check the network configuration.")

In [ ]:
# initialising the outputwriter to save data to excel files in the current folder. You can change this to .json, .csv, or .pickle as well
ow = timeseries.OutputWriter(net, output_path="./", output_file_type=".xlsx")
# adding vm_pu of all buses and line_loading in percent of all lines as outputs to be stored
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_load', 'p_mw')
ow.log_variable('res_load', 'q_mvar')


# starting the timeseries simulation for one day -> 96 15 min values.
timeseries.run_timeseries(net)
# now checkout the folders res_bus and res_line in your current working dir

In [98]:
# Load the results
vm_pu = pd.read_excel('res_bus/vm_pu.xlsx', index_col=0)
loading_percent = pd.read_excel('res_line/loading_percent.xlsx', index_col=0)
load_p_mw = pd.read_excel('res_load/p_mw.xlsx', index_col=0)
load_q_mvar = pd.read_excel('res_load/q_mvar.xlsx', index_col=0)


In [ ]:
import pandapower.plotting as plot
# Plot the grid
plot.simple_plot(net, show_plot=True)

In [ ]:
import matplotlib.pyplot as plt
# Generate the plots
plt.figure(figsize=(10, 5))
plt.plot(vm_pu, label=vm_pu.columns)
plt.xlabel('Time Step')
plt.ylabel('Voltage Magnitude (p.u.)')
plt.title('Voltage Magnitude over Time')
plt.legend()
plt.show()


In [ ]:

plt.figure(figsize=(10, 5))
plt.plot(loading_percent, label=loading_percent.columns)
plt.xlabel('Time Step')
plt.ylabel('Loading Percent')
plt.title('Line Loading over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(load_p_mw, label=load_p_mw.columns)
plt.xlabel('Time Step')
plt.ylabel('Active Power Load (MW)')
plt.title('Active Power Load over Time')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(load_q_mvar, label=load_q_mvar.columns)
plt.xlabel('Time Step')
plt.ylabel('Reactive Power Load (MVAr)')
plt.title('Reactive Power Load over Time')
plt.legend()
plt.show()

In [ ]:
import h5py

# Function to recursively explore groups and datasets
def explore_group(group, level=0):
    indent = "  " * level  # Indentation for better readability
    for key in group.keys():
        item = group[key]
        if isinstance(item, h5py.Group):
            print(f"{indent}Group: {key}")
            # Recursively explore the subgroup
            explore_group(item, level + 1)
        elif isinstance(item, h5py.Dataset):
            print(f"{indent}Dataset: {key}")
            print(f"{indent}  Shape: {item.shape}")
            print(f"{indent}  Data type: {item.dtype}")
            print(f"{indent}  First 5 lines of data:")
            print(item[:5])  # Adjust the slicing as needed
        else:
            print(f"{indent}Unknown item: {key}")

# Open the HDF5 file
file_path = "2020_weather.hdf5"  # Replace with the path to your HDF5 file
with h5py.File(file_path, "r") as hdf:
    print("Keys in the file:", list(hdf.keys()))
    
    # Access the 'WEATHER_SERVICE' group
    key = 'WEATHER_SERVICE'
    if key in hdf:
        group = hdf[key]
        print(f"Exploring '{key}' group recursively:")
        explore_group(group)
    else:
        print(f"Key '{key}' not found in the file.")

In [ ]:
import h5py
import pandas as pd

# Open the HDF5 file
file_path = "2020_weather.hdf5"  # Replace with the path to your HDF5 file
with h5py.File(file_path, "r") as hdf:
    # Access the dataset for apparent temperature
    dataset_path = "WEATHER_SERVICE/IN/WEATHER_APPARENT_TEMPERATURE_TOTAL/table"
    dataset = hdf[dataset_path]
    
    # Convert to pandas DataFrame
    data = pd.DataFrame(dataset[:])  # Convert structured array to DataFrame
    # Convert 'index' field (timestamp) to datetime
    data['index'] = pd.to_datetime(data['index'], unit='ns')
    
    # Filter for the first week of January 2020
    start_date = "2020-01-01"
    end_date = "2020-01-08"
    filtered_data = data[(data['index'] >= start_date) & (data['index'] < end_date)]
    
    # Export to CSV
    output_file = "temperatureWinter5m.csv"
    filtered_data.to_csv(output_file, index=False)
    print(f"Filtered data for the first week of January 2020 has been saved to '{output_file}'.")

In [ ]:
import h5py
import pandas as pd

# Open the HDF5 file
file_path = "2020_weather.hdf5"  # Replace with the path to your HDF5 file
with h5py.File(file_path, "r") as hdf:
    # Access the dataset for apparent temperature
    dataset_path = "WEATHER_SERVICE/IN/WEATHER_APPARENT_TEMPERATURE_TOTAL/table"
    dataset = hdf[dataset_path]
    
    # Convert to pandas DataFrame
    data = pd.DataFrame(dataset[:])  # Convert structured array to DataFrame
    # Convert 'index' field (timestamp) to datetime
    data['index'] = pd.to_datetime(data['index'], unit='ns')
    
    # Filter for the first week of January 2020
    start_date = "2020-01-01"
    end_date = "2020-01-08"
    filtered_data = data[(data['index'] >= start_date) & (data['index'] < end_date)]
    
    # Set the 'index' column as the DataFrame index
    filtered_data.set_index('index', inplace=True)
    
    # Resample to 15-minute intervals and calculate the mean
    resampled_data = filtered_data.resample('15T').mean()
    
    # Export to CSV
    output_file = "temperatureWinter15m.csv"
    resampled_data.to_csv(output_file)
    print(f"15-minute resolution data for the first week of January 2020 has been saved to '{output_file}'.")

In [ ]:
import h5py
import pandas as pd

# Open the HDF5 file
file_path = "2020_weather.hdf5"  # Replace with the path to your HDF5 file
with h5py.File(file_path, "r") as hdf:
    # Access the dataset for apparent temperature
    dataset_path = "WEATHER_SERVICE/IN/WEATHER_APPARENT_TEMPERATURE_TOTAL/table"
    dataset = hdf[dataset_path]
    
    # Convert to pandas DataFrame
    data = pd.DataFrame(dataset[:])  # Convert structured array to DataFrame
    # Convert 'index' field (timestamp) to datetime
    data['index'] = pd.to_datetime(data['index'], unit='ns')
    
    # Filter for the first week of January 2020
    start_date = "2020-01-01"
    end_date = "2020-01-08"
    filtered_data = data[(data['index'] >= start_date) & (data['index'] < end_date)]
    
    # Set the 'index' column as the DataFrame index
    filtered_data.set_index('index', inplace=True)
    
    # Resample to 1-hour intervals and calculate the mean
    resampled_data = filtered_data.resample('1H').mean()
    
    # Export to CSV
    output_file = "temperatureWinter1h.csv"
    resampled_data.to_csv(output_file)
    print(f"1-hour resolution data for the first week of January 2020 has been saved to '{output_file}'.")


In [ ]:
import pandas as pd

# Path to the CSV file
file_path = "heatpumpPrognosis.csv"  # Replace with the correct path to your CSV file

# Load the CSV into a DataFrame
df = pd.read_csv(file_path)

# Print the first few lines
print("First few lines of the 1-hour resolution data:")
print(df.head())
# Print the length of the DataFrame
print(f"The number of rows in the DataFrame: {len(df)}")

In [ ]:
import pandas as pd

# Path to your CSV file
file_path = "heatpumpPrognosis.csv"  # Replace with your actual file path

# Load the CSV into a DataFrame
df = pd.read_csv(file_path, sep=";")  # Adjust the separator if needed

# Clean and convert numeric columns safely
for col in ['meanP', 'stdP', 'meanQ', 'stdQ']:
    # Replace commas with dots and remove whitespace
    df[col] = (
        df[col]
        .astype(str)  # Ensure all values are strings
        .str.replace(r'\s+', '', regex=True)  # Remove spaces
        .str.replace(',', '.')  # Replace commas with dots
    )
    # Convert to float
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check for NaN values after conversion
if df[['meanP', 'stdP', 'meanQ', 'stdQ']].isnull().any().any():
    print("Warning: Some values could not be converted to numeric and are NaN.")

# Preserve the custom order of seasons
season_order = ['winter', 'summer', 'interim']
df['season'] = pd.Categorical(df['season'], categories=season_order, ordered=True)

# Group by 'season', 'dayOfWeek', and 'hour' (discard 'minute' for 1-hour resolution)
resampled_df = df.groupby(['season', 'dayOfWeek', 'hour']).mean().reset_index()

# Drop unnecessary columns
resampled_df = resampled_df.drop(columns=['minute', 'Unnamed: 0'], errors='ignore')

# Sort the DataFrame to ensure the order of seasons is maintained
resampled_df = resampled_df.sort_values(by=['season', 'dayOfWeek', 'hour'])

# Export the resampled data to a new CSV file
output_file = "heatpumpPrognosis1h.csv"
resampled_df.to_csv(output_file, sep=";", index=False)
print(f"Resampled data has been saved to '{output_file}'.")

In [ ]:
import pandas as pd

# Path to the CSV file
file_path = "temperatureWinter1h.csv"  # Replace with the correct path to your CSV file

# Load the CSV into a DataFrame
df = pd.read_csv(file_path)

# Print the first few lines
print("First few lines of the 1-hour resolution data:")
print(df.head())
# Print the length of the DataFrame
print(f"The number of rows in the DataFrame: {len(df)}")

In [44]:
T_amb = df['APPARENT_TEMPERATURE:TOTAL']

In [ ]:
print(T_amb)